In [1]:
import time, os, getpass
if 'set_dummy' not in locals():
    os.chdir('..')
set_dummy = 'dummy'
from sklearn.decomposition import PCA
import multiprocessing,threading,urllib,ipykernel,tempfile,concurrent.futures,matplotlib,warnings,zipfile # fix for fastai2.basics
from datasets import *
from architectures import *
from Learning.learning import *
from Learning.eval_metrics import *
from Learning.losses import *
from Utils.parser_ import get_args
from fastai2.basics import *
from fastai2.callback.all import *
from fastai2.vision.all import *

In [3]:
name = """
id:9_arch:h8E512_ds:lib+colo+notre_loss:tripletMargin_mpos:0.5_mneg:1.0_lr:3.0_maxsets:2000_sigmas-v:e011_thr:0.00016_WF:Hessian_PG:new_masks:AMOS-masks_tps:5000000_CamsB:5_fliprot_fewcams_ep:20_bs:9000_pos:2_dups:0
""".strip()
model = load_hardnet(name)
outname=name.split('_')[1].split(':')[1]
outname+='lib+colo+notre'
outname+='-NOfewcams-'
outname+='pca128'
outname+='lib'
print(outname)

Loading HardNet ...
arch: ['h8E512']
h8E512lib+colo+notre-NOfewcams-pca128lib


In [4]:
ds = torch.load('Datasets/6Brown/liberty.pt')
patches = ds['patches']
labels = ds['labels']

In [5]:
resize_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(32),
    transforms.ToTensor()
])

def chunks(lst, n): # Yield successive n-sized chunks from lst
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [6]:
inputs = [resize_transform(c) for c in patches]
inputs = torch.stack(inputs).cuda()

In [7]:
descs = []
for batch in chunks(inputs, 1024):
    descs += [model(batch).cpu().data.numpy()]
descs = np.concatenate(descs)

In [10]:
ncmps = 128
pca = PCA(n_components=ncmps)
pca.fit(descs)

PCA(copy=True, iterated_power='auto', n_components=128, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [11]:
w=torch.load('Models/{}/{}.pt'.format(name,name))
arch=w['model_arch']
w['model_arch']=arch+'pca'+str(ncmps) # change model arch
w['pca']=pca
w['pca_save_name'] = outname
torch.save(w, outname+'.pt')
print('saving to', outname+'.pt')
print('new model arch', w['model_arch'])

saving to h8E512lib+colo+notre-NOfewcams-pca128lib.pt
new model arch h8E512pca128
